<a href="https://colab.research.google.com/github/sumithdcosta/Python/blob/master/9.%20Methods/Super.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

9. Methods/Super.ipynb